In [0]:
from sedona.spark import *
from pyspark.sql.functions import col, expr, avg
from pyspark.sql.types import StructType, StructField, DoubleType

In [0]:

config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.3_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5'). \
    getOrCreate()

In [0]:
from sedona.spark import *

sedona = SedonaContext.create(config)

In [0]:
df_contours_lines = spark.table("geospatial.heights.contour_line").select( col("geometry"), col("property_value").alias("z"))


In [0]:
raster_envelope = df_contours_lines.selectExpr("ST_Envelope_Aggr(geometry) as geometry")
display(raster_envelope)

In [0]:
bounds = raster_envelope.selectExpr("ST_XMIN(geometry) AS xmin", "ST_XMAX(geometry) AS xmax", "ST_YMIN(geometry) AS ymin", "ST_YMAX(geometry) AS ymax").first()


display(bounds)


In [0]:
min_x = bounds.xmin
max_x = bounds.xmax
min_y = bounds.ymin
max_y = bounds.ymax

In [0]:
cell_size = 1000.0  # meters
cols = int((max_x - min_x) / cell_size)
rows = int((max_y - min_y) / cell_size)

In [0]:
schema = StructType([
    StructField("x", DoubleType()),
    StructField("y", DoubleType())
])

In [0]:
grid_data = [(min_x + col * cell_size, min_y + row * cell_size) 
             for row in range(rows) for col in range(cols)]
grid_df = spark.createDataFrame(grid_data, schema) \
    .withColumn("geom", expr("ST_Point(x, y)"))